 # Delta Lake tables 
 Use this notebook to explore Delta Lake functionality 

In [2]:
from pyspark.sql.types import StructType, IntegerType, StringType, DoubleType

# define the schema
schema = StructType() \
.add("ProductID", IntegerType(), True) \
.add("ProductName", StringType(), True) \
.add("Category", StringType(), True) \
.add("ListPrice", DoubleType(), True)

df = spark.read.format("csv").option("header","true").schema(schema).load("abfss://Test_retail@onelake.dfs.fabric.microsoft.com/fab_lakhouse.Lakehouse/Files")
# df now is a Spark DataFrame containing CSV data from "Files/products/products.csv".
display(df)

StatementMeta(, 00385afe-6239-44d2-a888-784f6abb1cf8, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5d3374dd-369a-47a8-a737-d4e990327993)

In [3]:
df.write.format("delta").saveAsTable("managed_products")

StatementMeta(, 00385afe-6239-44d2-a888-784f6abb1cf8, 5, Finished, Available, Finished)

In [4]:
%%sql
DESCRIBE FORMATTED managed_products;df.write.format("delta").saveAsTable("external_products", path="abfss://Test_retail@onelake.dfs.fabric.microsoft.com/fab_lakhouse.Lakehouse/Files/external_products")

StatementMeta(, 00385afe-6239-44d2-a888-784f6abb1cf8, 6, Finished, Available, Finished)

In [5]:
%%sql
DESCRIBE FORMATTED managed_products;

StatementMeta(, 00385afe-6239-44d2-a888-784f6abb1cf8, 7, Finished, Available, Finished)

<Spark SQL result set with 12 rows and 3 fields>

In [6]:
%%sql
DESCRIBE FORMATTED external_products;

StatementMeta(, 00385afe-6239-44d2-a888-784f6abb1cf8, 8, Finished, Available, Finished)

<Spark SQL result set with 12 rows and 3 fields>

In [7]:
%%sql
DROP TABLE managed_products;
DROP TABLE external_products;

StatementMeta(, 00385afe-6239-44d2-a888-784f6abb1cf8, 10, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [8]:
%%sql
CREATE TABLE products
USING DELTA
LOCATION 'Files/external_products';

StatementMeta(, 00385afe-6239-44d2-a888-784f6abb1cf8, 11, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [9]:
%%sql
SELECT * FROM products;

StatementMeta(, 00385afe-6239-44d2-a888-784f6abb1cf8, 12, Finished, Available, Finished)

<Spark SQL result set with 295 rows and 4 fields>

In [10]:
%%sql
UPDATE products
SET ListPrice = ListPrice * 0.9
WHERE Category = 'Mountain Bikes';

StatementMeta(, 00385afe-6239-44d2-a888-784f6abb1cf8, 13, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>

In [11]:
%%sql
DESCRIBE HISTORY products;

StatementMeta(, 00385afe-6239-44d2-a888-784f6abb1cf8, 14, Finished, Available, Finished)

<Spark SQL result set with 2 rows and 15 fields>

In [12]:
delta_table_path = 'Files/external_products'
# Get the current data
current_data = spark.read.format("delta").load(delta_table_path)
display(current_data)

# Get the version 0 data
original_data = spark.read.format("delta").option("versionAsOf", 0).load(delta_table_path)
display(original_data)

StatementMeta(, 00385afe-6239-44d2-a888-784f6abb1cf8, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, cf287c49-a981-49fa-84d8-410ade871633)

SynapseWidget(Synapse.DataFrame, 66910949-c5b7-465a-9ae8-1b3a1aa8c123)

In [13]:
%%sql
-- Create a temporary view
CREATE OR REPLACE TEMPORARY VIEW products_view
AS
    SELECT Category, COUNT(*) AS NumProducts, MIN(ListPrice) AS MinPrice, MAX(ListPrice) AS MaxPrice, AVG(ListPrice) AS AvgPrice
    FROM products
    GROUP BY Category;

SELECT *
FROM products_view
ORDER BY Category;    

StatementMeta(, 00385afe-6239-44d2-a888-784f6abb1cf8, 17, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 37 rows and 5 fields>

In [14]:
%%sql
SELECT Category, NumProducts
FROM products_view
ORDER BY NumProducts DESC
LIMIT 10;

StatementMeta(, 00385afe-6239-44d2-a888-784f6abb1cf8, 18, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 2 fields>

In [15]:
from pyspark.sql.functions import col, desc

df_products = spark.sql("SELECT Category, MinPrice, MaxPrice, AvgPrice FROM products_view").orderBy(col("AvgPrice").desc())
display(df_products.limit(6))

StatementMeta(, 00385afe-6239-44d2-a888-784f6abb1cf8, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7ab7ee14-743c-46d9-baad-0dd48bf59605)

# Streaming data

In [16]:
from notebookutils import mssparkutils
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Create a folder
inputPath = 'Files/data/'
mssparkutils.fs.mkdirs(inputPath)

# Create a stream that reads data from the folder, using a JSON schema
jsonSchema = StructType([
StructField("device", StringType(), False),
StructField("status", StringType(), False)
])
iotstream = spark.readStream.schema(jsonSchema).option("maxFilesPerTrigger", 1).json(inputPath)

# Write some event data to the folder
device_data = '''{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"error"}
{"device":"Dev2","status":"ok"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}'''

mssparkutils.fs.put(inputPath + "data.txt", device_data, True)

print("Source stream created...")

StatementMeta(, 00385afe-6239-44d2-a888-784f6abb1cf8, 20, Finished, Available, Finished)

Source stream created...


In [17]:
# Write the stream to a delta table
delta_stream_table_path = 'Tables/iotdevicedata'
checkpointpath = 'Files/delta/checkpoint'
deltastream = iotstream.writeStream.format("delta").option("checkpointLocation", checkpointpath).start(delta_stream_table_path)
print("Streaming to delta sink...")

StatementMeta(, 00385afe-6239-44d2-a888-784f6abb1cf8, 21, Finished, Available, Finished)

Streaming to delta sink...


In [18]:
%%sql
SELECT * FROM IotDeviceData;

StatementMeta(, 00385afe-6239-44d2-a888-784f6abb1cf8, 22, Finished, Available, Finished)

<Spark SQL result set with 9 rows and 2 fields>

In [19]:
# Add more data to the source stream
more_data = '''{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"error"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}'''

mssparkutils.fs.put(inputPath + "more-data.txt", more_data, True)

StatementMeta(, 00385afe-6239-44d2-a888-784f6abb1cf8, 23, Finished, Available, Finished)

True

In [20]:
%%sql
SELECT * FROM IotDeviceData;

StatementMeta(, 00385afe-6239-44d2-a888-784f6abb1cf8, 24, Finished, Available, Finished)

<Spark SQL result set with 16 rows and 2 fields>

In [21]:
deltastream.stop()

StatementMeta(, 00385afe-6239-44d2-a888-784f6abb1cf8, 25, Finished, Available, Finished)